<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/GET%20REAl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pandas_ta
!pip install tf_agents[reverb]
!pip install seaborn

In [4]:
pip install line_profiler

In [5]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
import reverb
import random
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import epsilon_greedy_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents.policies import greedy_policy
from tf_agents.policies.epsilon_greedy_policy import EpsilonGreedyPolicy
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.environments.batched_py_environment import BatchedPyEnvironment
from tf_agents.networks import categorical_q_network
from tf_agents.agents.categorical_dqn import categorical_dqn_agent

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

In [ ]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass


In [ ]:
num_iterations = 20000000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 1000  # @param {type:"integer"}

num_eval_episodes = 3  # @param {type:"integer"}
eval_interval = 100000  # @param {type:"integer"}

In [ ]:
def get_relative_change(df, column_name):
    relative_changes = []
    for i in range(len(df)):
        if i == 0:
            relative_changes.append(0)  # First element has no previous value
        else:
            relative_change = (df.iloc[i] - df.iloc[i-1]) / df.iloc[i-1]
            relative_changes.append(relative_change)
    return pd.DataFrame({column_name: relative_changes})

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/eurusd_hour.csv')
close = df['AC']
open = df['AO']
high = df['AH']
low = df['AL']
achange = df['ACh']
print(df)

In [ ]:
RSI = ta.rsi(close,14,scalar=1)
AROON = ta.aroon(high,low,14,scalar = 1)
AROON_UP = AROON['AROOND_14']
AROON_DOWN = AROON['AROONU_14']
CCI = ta.cci(high,low,close,14)
CCI = CCI.multiply(.001)
CCI = CCI.add(.5)
RVI = ta.rvi(close,high,low,14,scalar=1)
CHANGE = achange
CHANGE.name = 'Change'
lookback = 10
DIFF_enter = ta.sma(achange/np.std(achange),lookback,offset = -lookback)
DIFF_enter.name = 'DIFF_enter'

DIFF_hold = ta.sma(achange/np.std(achange),lookback,offset = -(lookback/2))
DIFF_hold.name = 'DIFF_hold'

INDICATORS = pd.concat([CHANGE,RSI,AROON_UP,AROON_DOWN,CCI,RVI,DIFF_enter,DIFF_hold],axis=1)

INDICATORS = INDICATORS.dropna(axis=0)
INDICATORS = INDICATORS.reset_index()

change = INDICATORS.pop('Change')
DIFF_enter = INDICATORS.pop('DIFF_enter')
DIFF_hold = INDICATORS.pop('DIFF_hold')
INDICATORS.pop('index')

INDICATORS = INDICATORS.to_numpy()
change = change.to_numpy()
change_dev = change.std()
change = change/change_dev
np.set_printoptions(suppress=True)

In [ ]:
print(np.min(DIFF_hold))

In [ ]:
plt.plot(INDICATORS)
plt.xlim(5400,5500)
#print(INDICATORS)

In [ ]:
def custom_formatter(x):
  return f'{x:.4f}'


def test_environment(env, num_steps):
    # Reset the environment
    observation = env.reset()

    # Get the upper bound of the action range
    action_spec = env.action_spec().maximum

    # Loop through the time steps and take random actions
    for _ in range(num_steps):
        # Select a random action from the range of valid values
        action = random.randint(0, action_spec)

        # Execute the action and get the next observation, reward, done, and info
        observation, reward, done, info = env.step(action)

        # Print the current time step
        np.set_printoptions(formatter={'float_kind':custom_formatter})
        print(action,"{:.4f}".format(reward),info[5:12])


In [ ]:
def cross(cur_diff,prev_diff):
  if cur_diff > 0 and prev_diff <= 0:
    return 5
  elif cur_diff < 0 and prev_diff >= 0:
    return 5
  else:
    return 0


In [ ]:
max_ep_len = 100

In [ ]:
def SortinoRatio(df, T):
    """Calculates the Sortino ratio from univariate excess returns.


    Args:
        df ([float]): The dataframe or pandas series of univariate excess returns.
        T ([integer]): The targeted return.
    """

    #downside deviation:

    #temp = np.minimum(0, df - T)**2
    temp = np.square(np.minimum(0, df - T))
    temp_expectation = np.mean(temp)
    downside_dev = np.sqrt(temp_expectation)

    if downside_dev == 0:
      return 0

    #Sortino ratio:

    sortino_ratio = np.mean(df - T) / downside_dev

    return(sortino_ratio)

In [ ]:
%load_ext line_profiler

In [ ]:
df = np.random.normal(loc=0, scale=1, size=100)
T = 0

In [ ]:
%lprun -f SortinoRatio SortinoRatio(df,T)

In [ ]:
actions = {
    0: lambda cur_change: {
        'value': cur_change,
        'direction': 1,
        'time': 0,
        'max_val': cur_change,
        'min_val': cur_change,
        'max_time': 0,
        'min_time': 0
    },
    1: lambda cur_change: {
        'value': -cur_change,
        'direction': -1,
        'time': 0,
        'max_val': -cur_change,
        'min_val': -cur_change,
        'max_time': 0,
        'min_time': 0
    },
    2: lambda cur_change: {
        'value': 0,
        'direction': 0,
        'time': 0,
        'max_val': 0,
        'min_val': 0,
        'max_time': 0,
        'min_time': 0
    }
    }

In [ ]:
class TradingEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int64, minimum=0, maximum=3, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(12,), dtype=np.float32, minimum=-100,maximum=100, name='observation')


    self._state = np.zeros(12)
    self._episode_ended = False
    self._count = random.randint(10000,60000)
    self._end_ep = self._count+max_ep_len
    self._balance = 0
    self._returns = np.zeros(0)
    self.trade_info = {
    'time': 0,
    'value': 0,
    'direction': 0,
    'max_val': 0,
    'min_val': 0,
    'max_time': 0,
    'min_time': 0
    }


  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def get_balance(self):
    return self._balance

  def _reset(self):

    self._state = np.zeros(12)
    self._episode_ended = False
    self._count = random.randint(10000,60000)
    self._end_ep = self._count+max_ep_len
    self._returns = np.zeros(0)
    self._balance = 0
    self.trade_info = {
    'time': 0,
    'value': 0,
    'direction': 0,
    'max_val': 0,
    'min_val': 0,
    'max_time': 0,
    'min_time': 0
    }

    return ts.restart(np.array(self._state, dtype=np.float32))

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()


    mult = 10
    diff_hold = DIFF_hold[self._count]
    profit = 0
    reward = 0
    cur_change = change[self._count+1]/mult
    cur_direction = self.trade_info['direction']
    action_func = actions.get(int(action), None)

    if action_func:
      profit = self.trade_info['value']
      reward = profit if profit >= 0 else max(-profit-.5,0)
      action_details = action_func(cur_change)
      self.trade_info.update(action_details)
    else:
      self.trade_info['value'] += cur_change * cur_direction
      self.trade_info['time'] += 0.01 if cur_direction != 0 else 0
      self.trade_info['max_val'] = max(self.trade_info['value'], self.trade_info['max_val'])
      self.trade_info['min_val'] = min(self.trade_info['value'], self.trade_info['min_val'])
      self.trade_info['max_time'] = 0 if self.trade_info['max_val'] == self.trade_info['value'] else self.trade_info['max_time'] + 0.01
      self.trade_info['min_time'] = 0 if self.trade_info['min_val'] == self.trade_info['value'] else self.trade_info['min_time'] + 0.01
      #reward = diff_hold*cur_direction


    self._balance += profit
    if profit != 0:
        self._returns = np.append(self._returns, profit)

    #info_list = np.fromiter(self.trade_info.values(), dtype=float)
    info_list = list(self.trade_info.values())

    #update state
    self._state = np.append(INDICATORS[self._count],info_list)

    if self._count >= self._end_ep:
      self._episode_ended = True
      np.append(self._returns,self.trade_info['value'])

    self._count += 1

    if self._episode_ended:

      reward = SortinoRatio(self._returns,0) if np.sum(self._returns) != 0 else -10
      return ts.termination(np.array(self._state, dtype=np.float32), reward=reward)
    else:
      return ts.transition(
          np.array(self._state, dtype=np.float32), reward=reward, discount=.9999)

In [ ]:
env = TradingEnv()


utils.validate_py_environment(env, episodes=2)

In [ ]:
train_py_env = env
eval_py_env = env
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [ ]:
import timeit

In [ ]:
timeit_result = timeit.timeit(lambda: test_environment(env, 1000), number=10)
print(timeit_result)

In [ ]:
print('Action Spec:')
print(env.action_spec())

In [ ]:
fc_layer_params = (20, 5)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1


In [ ]:
def activity_regularizer(multiplier, decay=0.99):
    # Initialize moving averages for mean and variance
    mean_avg = tf.Variable(0.0, trainable=False, name='mean_avg')
    variance_avg = tf.Variable(1.0, trainable=False, name='variance_avg')

    def regularizer(y_pred):
        # Calculate the current mean and variance
        current_mean = tf.reduce_mean(y_pred)
        current_variance = tf.math.reduce_variance(y_pred)

        # Update moving averages
        update_mean = tf.compat.v1.assign(mean_avg, decay * mean_avg + (1 - decay) * current_mean)
        update_variance = tf.compat.v1.assign(variance_avg, decay * variance_avg + (1 - decay) * current_variance)

        with tf.control_dependencies([update_mean, update_variance]):
            # Use the moving averages for regularization
            mean = mean_avg
            variance = variance_avg

        return multiplier * (tf.square(mean - 0) + tf.square(variance - 1))

    return regularizer

In [ ]:


#import base64
#import imageio
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

from tf_agents.agents.dqn.dqn_agent import DqnAgent, DdqnAgent
from tf_agents.networks.q_network import QNetwork

from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment

from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.replay_buffers.tf_uniform_replay_buffer import TFUniformReplayBuffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

# Globals
NUMBER_EPOSODES = 1000000
COLLECTION_STEPS = 1
BATCH_SIZE = 32
EVAL_EPISODES = 3
EVAL_INTERVAL = 1000



In [ ]:
print('Observation Spec:')
print(train_env.time_step_spec().observation)

print('Reward Spec:')
print(train_env.time_step_spec().reward)

print('Action Spec:')
print(train_env.action_spec())

train_env = tf_py_environment.TFPyEnvironment(env)
evaluation_env = tf_py_environment.TFPyEnvironment(env)

In [ ]:
from tf_agents.networks import network


class CustomQNetwork(network.Network):
    def __init__(self, input_tensor_spec, action_spec, name='CustomQNetwork'):
        super(CustomQNetwork, self).__init__(input_tensor_spec=input_tensor_spec,
                                             state_spec=(),
                                             name=name)
        # Define your custom layers and architecture here
        self.dense1 = layers.Dense(100,activation='gelu', activity_regularizer=activity_regularizer(0.01))
        self.drop1 = layers.Dropout(.2)
        self.dense2 = layers.Dense(100,activation='gelu', activity_regularizer=activity_regularizer(0.01))
        self.drop2 = layers.Dropout(.2)
        self.dense2 = layers.Dense(100,activation='gelu', activity_regularizer=activity_regularizer(0.01))
        self.drop2 = layers.Dropout(.2)
        self.q_values_layer = tf.keras.layers.Dense(action_spec.maximum - action_spec.minimum + 1,
                                                    activation=None)

    def call(self, observation, step_type=None, network_state=(), training=False):
        # Pass the observation through your custom layers
        x = self.dense1(observation)
        x = self.drop1(x)
        x = self.dense2(x)
        x = self.drop2(x)
        q_values = self.q_values_layer(x)
        return q_values, network_state

In [ ]:

class CategoricalQNetwork(network.Network):
    def __init__(self, input_tensor_spec, action_spec, num_atoms=51,name='CategoricalQNetwork'):
        super(CategoricalQNetwork, self).__init__(input_tensor_spec=input_tensor_spec,
                                                  state_spec=(),
                                                  name=name)
        # Check if action_spec is a BoundedTensorSpec
        if not isinstance(action_spec, tensor_spec.BoundedTensorSpec):
            raise TypeError('action_spec must be a BoundedTensorSpec. Got: %s' % (action_spec,))

        self._num_actions = action_spec.maximum - action_spec.minimum + 1
        self._num_atoms = num_atoms

        # Create a BoundedTensorSpec for the q_network action
        q_network_action_spec = tensor_spec.BoundedTensorSpec((), tf.int32,
                                                              minimum=0,
                                                              maximum=self._num_actions * num_atoms - 1)

        # Create the q_network
        self._q_network = CustomQNetwork(input_tensor_spec=input_tensor_spec,
                                             action_spec=q_network_action_spec,
                                             )
    def call(self, observation, step_type=None, network_state=(), training=False):
        logits, network_state = self._q_network(observation, step_type, network_state, training=training)
        logits = tf.reshape(logits, [-1, self._num_actions, self._num_atoms])
        return logits, network_state
    @property
    def num_atoms(self):
        return self._num_atoms

In [ ]:
observation_spec = train_env.observation_spec()  # Specify the input tensor spec for your network
action_spec = train_env.action_spec()
num_atoms = 5
dropout_rate = 0.2
custom_network = CategoricalQNetwork(observation_spec,action_spec, num_atoms)

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=custom_network,
    optimizer=optimizer,
    min_q_value=-3,
    max_q_value=3,
    n_step_update=2,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=1,
    train_step_counter=train_step_counter)
agent.initialize()


In [ ]:

def get_average_reward(environment, policy, episodes=10):

    total_reward = 0.0

    for _ in range(episodes):
        time_step = environment.reset()
        episode_reward = 0.0

    while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = environment.step(action_step.action)
        episode_reward += time_step.reward

    total_reward += episode_reward
    avg_reward = total_reward / episodes

    return avg_reward.numpy()[0]



In [ ]:
class ExperienceReplay(object):
    def __init__(self, agent, enviroment):
        self._replay_buffer = TFUniformReplayBuffer(
            data_spec=agent.collect_data_spec,
            batch_size=enviroment.batch_size,
            max_length=10000)

        self._random_policy = RandomTFPolicy(train_env.time_step_spec(),
                                                enviroment.action_spec())

        self._fill_buffer(train_env, self._random_policy, steps=100)

        self.dataset = self._replay_buffer.as_dataset(
            num_parallel_calls=tf.data.AUTOTUNE,
            sample_batch_size=BATCH_SIZE,
            num_steps=2).prefetch(tf.data.AUTOTUNE)

        self.iterator = iter(self.dataset)



    def _fill_buffer(self, enviroment, policy, steps):
        for _ in range(steps):
            self.timestamp_data(enviroment, policy)

    def timestamp_data(self, environment, policy):
        time_step = environment.current_time_step()
        action_step = policy.action(time_step)
        next_time_step = environment.step(action_step.action)
        timestamp_trajectory = trajectory.from_transition(time_step, action_step, next_time_step)

        self._replay_buffer.add_batch(timestamp_trajectory)

In [ ]:
agent.train = common.function(agent.train)

def train(agent):
    experience_replay = ExperienceReplay(agent, train_env)

    agent.train_step_counter.assign(0)

    avg_reward = get_average_reward(eval_env, agent.policy, EVAL_EPISODES)
    rewards = [avg_reward]

    for _ in range(NUMBER_EPOSODES):

        for _ in range(COLLECTION_STEPS):
            experience_replay.timestamp_data(train_env, agent.collect_policy)

        experience, info = next(experience_replay.iterator)
        train_loss = agent.train(experience).loss

        if agent.train_step_counter.numpy() % EVAL_INTERVAL == 0:
            avg_reward = get_average_reward(eval_env, agent.policy, EVAL_EPISODES)
            print('Episode {0} - Average reward = {1}, Loss = {2}.'.format(
					agent.train_step_counter.numpy(), avg_reward, train_loss))
            rewards.append(avg_reward)

    return rewards


print("**********************************")
print("Training DDQN")
print("**********************************")
ddqn_reward = train(agent)

In [ ]:
def compute_avg_return_print(environment, policy, num_episodes=10):

  total_return = 0.0
  for episode in range(num_episodes):
    balance_arr = []
    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      obs = time_step.observation.numpy()

      balance = env.get_balance()
      balance_arr.append(balance)
      # Print the episode number and observation at each step
      print(f"Episode {episode + 1}: Observation = {obs}")

      episode_return += time_step.reward
    plt.plot(balance_arr)
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

In [ ]:
avg_reward = get_average_reward(eval_env, agent.policy, 1)
print(avg_reward)

In [1]:
avg_return = compute_avg_return_print(eval_env, agent.policy, 20)
print(avg_return)

NameError: ignored

In [ ]:
weights = custom_network.get_weights()
print(weights)